In [10]:
import cv2
import os
import re
import pickle
import matplotlib.pyplot as plt
import tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
lyric_set = []
pixels_set = []
path = "./dataset"
dir_list = os.listdir(path)

n = 2
for album in dir_list:
    try:
        path_album = "./dataset"
        path_album += str("/" + album)
        dir_album = os.listdir(path_album)
        cover_path = path_album
        for img in dir_album:
            if ".jpg" in img:
                cover_path += "/" + img
        for song in dir_album:
            if ".txt" in song:
                song_path = path_album + "/" + song
                with open(song_path) as song_lyrics:
                    lines = song_lyrics.readlines()
                    lyric_set.append(lines)
                cover_pixels = cv2.imread(cover_path)
                pixels_set.append(cover_pixels)      
    except:
        pass

In [3]:
lyric_set = [[lyric[:-1] for lyric in lyrics] for lyrics in lyric_set]
lyric_set = [[lyric for lyric in lyrics if lyric != ''] for lyrics in lyric_set]

In [4]:
lyric_set[1]

['My mind is full bursting over',
 "With all these things I can't remember",
 'Every little single memory reminds me of you',
 'My eyes were weary with all these tears',
 'You left your shadow in my dreams',
 'And all my doubts seemed to disappear when you came along',
 'Flowers melting up into the sky',
 'Hear my heart where our love collides',
 'We hear the songs we found in the times we lost our way',
 'Gentle memories replace our tears',
 'All the love we had is still right here',
 'We hear the songs we found in the time we lost our way',
 'From without words can not describe',
 'What caused the stars to fall deep inside',
 'Every little single memory reminds me of you',
 'Our days are gone lost forever',
 'Reflecting light glistening under water',
 'Naturally this could be everything that seems so unreal',
 'Flowers melting up into the sky',
 'Hear my heart where our love collides',
 'We hear the songs we found in the time we lost our way',
 'Gentle memories replace our tears',
 '

In [5]:
print(len(pixels_set))
print(len(lyric_set))

24692
24692


In [7]:
full_set = [lyric_set, pixels_set]

[['Instrumental'],
 ['My mind is full bursting over',
  "With all these things I can't remember",
  'Every little single memory reminds me of you',
  'My eyes were weary with all these tears',
  'You left your shadow in my dreams',
  'And all my doubts seemed to disappear when you came along',
  'Flowers melting up into the sky',
  'Hear my heart where our love collides',
  'We hear the songs we found in the times we lost our way',
  'Gentle memories replace our tears',
  'All the love we had is still right here',
  'We hear the songs we found in the time we lost our way',
  'From without words can not describe',
  'What caused the stars to fall deep inside',
  'Every little single memory reminds me of you',
  'Our days are gone lost forever',
  'Reflecting light glistening under water',
  'Naturally this could be everything that seems so unreal',
  'Flowers melting up into the sky',
  'Hear my heart where our love collides',
  'We hear the songs we found in the time we lost our way',


In [13]:
with open('dataset.pkl','wb') as f:
    pickle.dump(full_set, f)